In [ ]:
import sys
import os

# Add project root to pythonpath
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import zscore
import seaborn as sns

from src.data.load_data import load_data

In [ ]:
# Load data
data_path = 'data/train/train.csv'
data = load_data(data_path)

In [ ]:
data.describe()

In [ ]:
#data.head()
#data.info()

In [ ]:
data_train = data.copy()

In [ ]:
#Let's analize those columns with null values

# Columns with null values
null_columns = data_train.isnull().sum().sort_values(ascending=False)
null_columns = null_columns[null_columns > 0]

# Temp DataFrame of columns with null values
output_table = pd.DataFrame({
    "Column": null_columns.index,
    "NullCount": null_columns.values,
    "DataType": [data_train[col].dtype for col in null_columns.index],
    "UniqueValues": [
        ", ".join(map(str, data_train[col].dropna().unique())) if data_train[col].dtype == "object" else None
        for col in null_columns.index
    ],
    "CorrelationWithSalePrice": [
        data_train[col].corr(data_train["SalePrice"]) if data_train[col].dtype != "object" else None
        for col in null_columns.index
    ], 
})

output_table.reset_index(drop=True, inplace=True)
output_table

In [ ]:
# Let's plot string columns with null values against the selling price

string_columns = data_train.select_dtypes(include=["object"])
string_columns_with_nulls = [col for col in string_columns.columns if data_train[col].isnull().any()]

# Assign 'No' as default value
data_train[string_columns_with_nulls] = data_train[string_columns_with_nulls].fillna("No")

# Plot value vs SalePrice
for column in string_columns_with_nulls:
    sns.catplot(
        x=column,
        y="SalePrice",
        data=data_train,
        kind="box",
        aspect=1,
    )
    plt.title(f"{column} vs SalePrice")
    plt.tight_layout()
    plt.show()

# We'll drop: 'Alley','MasVnrType','BsmtCond','PoolQC','Fence','MiscFeature','GarageQual', 'GarageCond' and 'BsmtFinType2'

In [ ]:
def plot_corr_heatmap(data, annot=False, full_size=False, corr_threshold = None):
    if full_size:
        plt.figure(figsize=(20,12))

    # Only numeric columns are selected
    numeric_data = data.select_dtypes(include=["number"])
    correlation_matrix = numeric_data.corr()
    
    annot_data = annot
    if annot and corr_threshold:
        annot_mask = correlation_matrix.abs() >= corr_threshold
        annot_data = np.where(annot_mask, correlation_matrix.round(2), "")
    
    sns.heatmap(
        correlation_matrix, 
        annot=annot_data if annot else False, 
        center=0, 
        cmap="coolwarm", 
        fmt="s",
        linewidth=.5 
    )
    
    #plt.show()

In [ ]:
def getDatasetNumericColumns(dataset):
    return dataset.select_dtypes(include=['number']).columns.tolist()

In [ ]:
numeric_columns = getDatasetNumericColumns(data)

In [ ]:
numeric_columns_wo_price = numeric_columns.copy()
numeric_columns_wo_price.remove("SalePrice")
numeric_columns_wo_price.remove("Id")

In [ ]:
len(data_train)

In [ ]:
# Plot config
num_columns = len(numeric_columns_wo_price)
rows = (num_columns // 3) + (num_columns % 3 > 0)  # Get rows needed
fig, axes = plt.subplots(rows, 3, figsize=(15, 5 * rows))  # Set the plot size

# Adjust layout to avoid overlapping
fig.tight_layout(pad=5.0)

# Get the total number of rows in the dataset
total_rows = len(data_train)

# Generate scatter plots for each numeric column against SalePrice
for i, column in enumerate(numeric_columns_wo_price):
    row = i // 3
    col = i % 3
    ax = axes[row, col] if rows > 1 else axes[col]

    # Count non-null rows in the current column
    non_null_count = data_train[column].notna().sum()

    # Plot the scatter plot
    ax.scatter(data_train[column], data_train["SalePrice"], alpha=0.6)
    ax.set_title(f'{column} vs SalePrice\n({round(non_null_count * 100 / total_rows ,2)})')
    ax.set_xlabel(column)
    ax.set_ylabel("SalePrice")

# Hide empty plots if overlaping
for j in range(i + 1, rows * 3):
    row = j // 3
    col = j % 3
    ax = axes[row, col] if rows > 1 else axes[col]
    ax.axis("off")

# Adjust spacing between plots
plt.subplots_adjust(hspace=0.5)
plt.show()

# Remove rows based on specific limit criteria to clean the dataset
    # LotFrontage > 300 [934, 1298] 
    # LotArea > 70000 [249, 313, 335, 451, 706]
    # OverallCond [378, 691, 1182]
    # MasVnrArea > 1300 [297, 1169]
    # houseAge > 100 and SalePrice > 400000 [185]
    # totalBaths >= 5 [738, 921]
    # MiscVal >= 4000 [346, 1230]
    # totalCoveredArea >= 6000 [496, 523]

# Corresponding to rows = [934, 1298, 249, 313, 335, 451, 706, 378, 691, 1182, 297, 1169, 185, 738, 921, 346, 1230, 496, 523]
# Remove PoolArea as it has too little information

In [ ]:
#data_train.query('totalCoveredArea >= 6000')

In [ ]:
# Add new features to the dataset
def add_features(dataset):
    dataset['houseAge'] = dataset['YrSold'] - dataset['YearBuilt']
    dataset['houseRemodAge'] = dataset['YrSold'] - dataset['YearRemodAdd']
    #dataset['garageAge'] = dataset['YrSold'] - dataset['GarageYrBlt']
    dataset['totalBaths'] = dataset['BsmtFullBath'] + dataset['FullBath'] + 0.5* (dataset['HalfBath'] + dataset['BsmtHalfBath'])
    dataset['porchDeckArea'] = dataset['WoodDeckSF'] + dataset['OpenPorchSF'] + dataset['EnclosedPorch'] + dataset['3SsnPorch'] + dataset['ScreenPorch']
    #dataset['totalFinArea'] = dataset['BsmtFinSF1'] + dataset['BsmtFinSF2'] + dataset['1stFlrSF'] + dataset['2ndFlrSF']
    dataset['totalCoveredArea'] = dataset['GrLivArea'] + dataset['TotalBsmtSF']

    return dataset

In [ ]:
# Drop unnecessary features from the dataset
def drop_features(dataset):
    # Drop columns with little information or high correlation with others
    dataset = dataset.drop(columns=['Id','Alley','MasVnrType','BsmtCond','PoolQC','Fence',
                                    'MiscFeature','GarageQual','GarageCond', 'BsmtFinType2'])
    
    # Drop columns used to create new features
    dataset = dataset.drop(columns=['YrSold','YearBuilt','YearRemodAdd','BsmtFullBath',
                                   'FullBath','HalfBath','BsmtHalfBath','WoodDeckSF',
                                   'OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch',
                                    'BsmtFinSF1','BsmtFinSF2','1stFlrSF','2ndFlrSF','GrLivArea',
                                   'TotalBsmtSF','GarageYrBlt','GarageArea'])

    return dataset

In [ ]:
def fill_null_values(dataset):
    string_columns_with_nulls = ['FireplaceQu','GarageFinish','GarageType',
                                 'BsmtExposure','BsmtFinType1','BsmtQual','Electrical']
    dataset[string_columns_with_nulls] = dataset[string_columns_with_nulls].fillna("No")
    
    numeric_columns_with_nulls = ['LotFrontage','MasVnrArea']
    dataset[numeric_columns_with_nulls] = dataset[numeric_columns_with_nulls].fillna(0)

    return dataset

In [ ]:
def feature_engineering(dataset):
    dataset_final = dataset.copy()
    dataset_final = add_features(dataset_final)
    dataset_final = drop_features(dataset_final)
    dataset_final = fill_null_values(dataset_final)
    
    return dataset_final

In [ ]:
data_train = feature_engineering(data)

In [ ]:
getDatasetNumericColumns(data_train)

In [ ]:
#plot_corr_heatmap(data_train[['totalFinArea', 'totalCoveredArea','SalePrice']], True, False, 0.5)
#plot_corr_heatmap(data_train[['GarageArea', 'GarageCars','SalePrice']], True, False, 0.5)

# Drop GarageArea and GarageCars (both are always simultaneously informed; prefer GarageCars for its categorical nature)

In [ ]:
# Plot correlation heatmap for numeric columns with a threshold
plot_corr_heatmap(data_train, True, True, 0.8)

# Drop GarageArea and GarageCars (previously checked both are always simultaniously informed)

In [ ]:
# SalePrice distribution
sns.histplot(data_train["SalePrice"], kde=True, alpha=0.7)

In [ ]:
# Normalize SalePrice distribution 
sns.histplot(np.log1p(data_train["SalePrice"]), kde=True, alpha=0.7)

In [ ]:
# Identify square feet columns and analyze their correlations
data_train_columns = data_train.select_dtypes(include=['number']).columns.tolist()
square_feet_columns = [col for col in data_train_columns if 'sf' in col.lower()] 
area_columns = [col for col in data_train_columns if 'area' in col.lower()] 
sq_area_columns = square_feet_columns + area_columns


In [ ]:
#plot_corr_heatmap(data_train[['LotArea', 'totalCoveredArea', 'porchDeckArea', 'totalFinArea']])

In [ ]:
plot_corr_heatmap(data_train[area_columns])

In [ ]:
plot_corr_heatmap(data_train[sq_area_columns])

In [ ]:
plot_corr_heatmap(data_train[square_feet_columns])